In [ ]:
# Cloud Shell
gcloud auth list

In [ ]:
export REPO=valkyrie-docker
export DCKR_IMG=valkyrie-dev
export TAG=valkyrie-dev

In [ ]:
export REGION=us-east1
export ZONE=us-east1-c

##Task 1. Create a Docker image and store the Dockerfile

In [ ]:
# Sourcing setup script
source <(gsutil cat gs://cloud-training/gsp318/marking/setup_marking_v2.sh)

In [ ]:
# Downloading and extracting application
gsutil cp gs://spls/gsp318/valkyrie-app.tgz .
tar -xzf valkyrie-app.tgz
cd valkyrie-app

In [ ]:
# Creating Dockerfile
cat > Dockerfile <<EOF
FROM golang:1.10
WORKDIR /go/src/app
COPY source .
RUN go install -v
ENTRYPOINT ["app","-single=true","-port=8080"]
EOF

In [ ]:
# Building Docker image
docker build -t $DCKR_IMG:$TAG .

In [ ]:
# Executing Step 1 script
cd ..
bash ~/marking/step1_v2.sh

##Task 2. Test the created Docker image

In [ ]:
# Running Docker container
cd valkyrie-app
docker run -d -p 8080:8080 $DCKR_IMG:$TAG

In [ ]:
# Executing Step 2 script
cd ..
bash ~/marking/step2_v2.sh

##Task 3. Push the Docker image to the Artifact Registry


In [ ]:
cd valkyrie-app

In [ ]:
# Creating Artifact Repository
gcloud artifacts repositories create $REPO \
    --repository-format=docker \
    --location=$REGION \
    --description="awesome lab" \
    --async

In [ ]:
# Configuring Docker authentication
gcloud auth configure-docker $REGION-docker.pkg.dev --quiet

In [ ]:
# Tagging and pushing Docker image
# Re-tag, the format should resemble the following: LOCATION-docker.pkg.dev/PROJECT-ID/REPOSITORY/IMAGE
Image_ID=$(docker images --format='{{.ID}}')

In [ ]:
docker tag $Image_ID $REGION-docker.pkg.dev/$DEVSHELL_PROJECT_ID/$REPO/$DCKR_IMG:$TAG

In [ ]:
docker push $REGION-docker.pkg.dev/$DEVSHELL_PROJECT_ID/$REPO/$DCKR_IMG:$TAG

##Task 4. Create and expose a deployment in Kubernetes

In [ ]:
# Updating Kubernetes deployment
sed -i s#IMAGE_HERE#$REGION-docker.pkg.dev/$DEVSHELL_PROJECT_ID/$REPO/$DCKR_IMG:$TAG#g k8s/deployment.yaml

In [ ]:
# Configuring Kubernetes cluster
gcloud container clusters get-credentials valkyrie-dev --zone $ZONE

In [ ]:
# Deploying application to Kubernetes
kubectl create -f k8s/deployment.yaml
kubectl create -f k8s/service.yaml